This script is convert raw residential permit data to import ready PSRC data 

In [11]:
import pandas as pd 
import numpy as np
import os
os.getcwd()

'C:\\Users\\Angela\\Documents\\GitHub\\Latest_Github\\permits\\scripts'

In [12]:
working_dir = 'J:/Projects/Permits/17Permit/data/2working/'

In [13]:
temp = pd.read_csv(working_dir + 'template.csv')
raw_data = pd.read_csv(working_dir + 'KENT.csv')

In [14]:
temp.columns

Index([u'PSRCIDN', u'PERMITNO', u'SORT', u'MULTIREC', u'PIN', u'ADDRESS',
       u'HOUSENO', u'PREFIX', u'STRNAME', u'STRTYPE', u'SUFFIX', u'UNIT_BLD',
       u'ZIP', u'ISSUED', u'FINALED', u'STATUS', u'TYPE', u'PS', u'UNITS',
       u'BLDGS', u'LANDUSE', u'CONDO', u'VALUE', u'ZONING', u'NOTES',
       u'NOTES2', u'NOTES3', u'NOTES4', u'NOTES5', u'NOTES6', u'NOTES7',
       u'LOTSIZE', u'PIN_PARENT'],
      dtype='object')

In [15]:
raw_data.head(2)

Permit ID#              Project Name    Tax PID#        Site Address  \
0  CNST-2162979  REYNOLDS RESIDENCE - ADU  4030500130    13238 SE 261 ST    
1  CNST-2161682      COUL RESIDENCE - ADU  1858600016  26412  106 AVE SE    

   Site ZipCode   Issue Date   Final Date Status         # of Units  \
0         98042   11/18/2016   07/18/2017   DONE                      
1         98030   12/19/2016   11/28/2017   DONE                      

         # of Buildings Est. Value          Condo Lot Area/Size Zoning Code  \
0                          $167.93                      15,188         SR-6   
1                            $0.00                       6,869         SR-6   

                                    Work Description Structure Type  \
0  CONSTRUCT DETACHED ADU WHICH WILL CONSIST OF 1...            ADU   
1  BUILDING, PLUMBING, MECHANICAL PERMIT         ...            ADU   

  Permit Activity                 Land Use  
0        Addition  Accessory Dwelling Unit  
1        Addition  Accessory Dwelling Unit

In [16]:
test1 = raw_data 

In [17]:
# import the permit number and address column name 
my_permit_id = 'Permit ID#' #the column name for permit id from raw data
my_address = 'Site Address'

# Sort 

Creat Sort ID by following previous format. The format could be different from city to city.  

In [18]:
test1['SORT'] = ''
for i in range(len(test1)): 
    p = test1['Permit ID#'][i]
    p1 = '2017' + p[:5] + '0' + p[-4:]
    test1['SORT'][i] = p1

c:\anaconda\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
test1['SORT'].head()

0    2017CNST-02979
1    2017CNST-01682
2    2017CNST-04320
3    2017CNST-01580
4    2017CNST-01791
Name: SORT, dtype: object

# Site Address

split full address into ['HOUSENO', 'PREFIX', 'STRNAME', 'STRTYPE', 'SUFFIX', 'UNIT_BLD', 'CITY', 'STATE', 'ZIP'] columns

In [20]:
test1[my_address].head(10)

0      13238 SE 261 ST 
1    26412  106 AVE SE 
2      24602  98 AVE S 
3       420 W SMITH ST 
4    3900  VETERANS DR 
5    3900  VETERANS DR 
6    3900  VETERANS DR 
7    3900  VETERANS DR 
8    3900  VETERANS DR 
9    3900  VETERANS DR 
Name: Site Address, dtype: object

In [21]:
col1 = ['HOUSENO', 'PREFIX', 'STRNAME', 'STRTYPE', 'SUFFIX', 'UNIT_BLD', 'CITY', 'STATE', 'ZIP']
for c in col1:
    test1[c] = ''

In [22]:
# Roughly split full adress into columns, without QA/QC
for i in range(len(test1)):
    test1[my_address][i] = test1[my_address][i].replace(',', ' ')
    address = test1[my_address][i].split()
    if len(address) <= 9:
        for j in range(len(address)): 
            test1[col1[j]][i] = address[j]
    else: 
        print address 
        #for j in range(4): #why 4? already investigated (the default is 6)
            #test1[col1[j]][i] = address[j+2]

c:\anaconda\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
c:\anaconda\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
test1[['HOUSENO', 'PREFIX', 'STRNAME', 'STRTYPE', 'SUFFIX', 'UNIT_BLD', 'CITY', 'STATE', 'ZIP']][-20:]

HOUSENO     PREFIX STRNAME STRTYPE SUFFIX UNIT_BLD CITY STATE ZIP
492     520     SCENIC     WAY       S                               
493   26215        143     AVE      SE                               
494   13927         SE     238      PL                               
495   12524         SE     264      ST                               
496    4520          S     220      ST                               
497   10208         SE     228      ST                               
498     635  CLEARVIEW   POINT                                       
499   13523         SE     250      ST                               
500   14616         SE     284      ST                               
501   28235        104     AVE      SE                               
502   25124        132     AVE      SE                               
503   20510         93      PL       S                               
504   20610         93      PL       S                               
505   20609         93      PL       S                               
506   20602         93      PL       S                               
507   20616         93      PL       S                               
508   20615         93      PL       S                               
509   20703         93      PL       S                               
510     733          5     AVE       S                               
511     733          5     AVE       S

In [24]:
np.unique(test1[[ 'STRTYPE']])

array(['', 'AVE', 'CT', 'KANGLEY', 'LN', 'N', 'PL', 'S', 'SE', 'ST', 'WAY'], dtype=object)

In [25]:
np.unique(test1[[ 'STRNAME']])

array(['192', '196', '197', '202', '204', '208', '212', '218', '219',
       '220', '222', '227', '228', '229', '238', '240', '241', '242',
       '243', '244', '245', '246', '248', '250', '251', '252', '253',
       '255', '259', '260', '261', '264', '266', '267', '269', '270',
       '272', '273', '276', '277', '278', '280', '284', '292', '293',
       '294', '295', '297', 'ALDER', 'AVE', 'CT', 'DR', 'GUIBERSON', 'HWY',
       'KENT', 'LN', 'NADEN', 'PL', 'POINT', 'RD', 'SMITH', 'ST', 'TITUS',
       'WAY'], dtype=object)

In [26]:
list_prefix = ['S', 'SE', 'SW', 'W', 'NW', 'N','NE','E']
list_suffix = ['S', 'SE', 'SW', 'W', 'NW', 'N','NE','E']

In [27]:
# Fix the prefix 
for i in range (len(test1)): 
    if test1['PREFIX'][i] not in list_prefix: 
        # move the value into next column 
        test1.loc[i:i,['PREFIX', 'STRNAME', 'STRTYPE', 'SUFFIX', 'UNIT_BLD']] = test1.loc[i:i,['PREFIX', 'STRNAME', 'STRTYPE', 'SUFFIX', 'UNIT_BLD']].shift(1,axis=1)
    else:
        continue 

In [28]:
city_list = ['Auburn', 'AUBURN', 'RENTON', 'ENUMCLAW', 'Enumclaw', 'BEND', 'CARNATION', 
            'DUVALL', 'ISSAQUAH', 'Issaquah', 'KENT', 'King', 'PRESTON', 'RAVENSDALE',
            'REDMOND', 'Renton', 'SAMMAMISH', 'SNOQUALMIE', 'VASHON', 'WOODINVILLE',
            ]


street_type = ['AVE', 'BLVD', 'CIR','CIRCLE','CROSSING', 'COVE', 'CT', 'DR', 'HWY', 
               'LN', 'LINE', 'LOOP', 'PL', 'RD', 'ROAD','Road','SHORE',
               'ST', 'TRAIL', 'TRL',  'WAY']

In [29]:
backup_test1 = test1 # just in case, if I lost all data later 

In [30]:
# use ['SUFFIX'] as the base to inspect the row # data, QA/QC street name and unit building name.
for i in range(len(test1)): 
    #print test1['SUFFIX'][i]
    if len(test1[my_address][i].split()) > 4: 
        #print i 
        if str(test1['SUFFIX'][i]) in street_type: 
            test1['STRNAME'][i] = test1['STRNAME'][i] + ' ' + test1['STRTYPE'][i] 
            test1.loc[i:i,['STRTYPE', 'SUFFIX', 'UNIT_BLD']] = test1.loc[i:i,['STRTYPE', 'SUFFIX', 'UNIT_BLD']].shift(-1,axis=1)
        else: 
            if str(test1['SUFFIX'][i]) in list_suffix: # suffix is suffix
                continue
            if str(test1['SUFFIX'][i]) in city_list: # city name in suffix
                test1.loc[i:i,['SUFFIX', 'UNIT_BLD']] = test1.loc[i:i,['SUFFIX', 'UNIT_BLD']].shift(1,axis=1)
            if str(test1['SUFFIX'][i]) in ['WA']: # state name in suffix
                test1.loc[i:i,['SUFFIX', 'UNIT_BLD']] = test1.loc[i:i,['SUFFIX', 'UNIT_BLD']].shift(2,axis=1)
            if str(test1['SUFFIX'][i])[0] == 9: # zipcode in suffix
                test1.loc[i:i,['SUFFIX', 'UNIT_BLD']] = test1.loc[i:i,['SUFFIX', 'UNIT_BLD']].shift(2,axis=1)
            else: #may be street name in suffix ? or nothing in suffix 
                #print test1['ADDRESS'][i]
                continue
            #test1.loc[i:i,['SUFFIX', 'UNIT_BLD']] = test1.loc[i:i,['SUFFIX', 'UNIT_BLD']].shift(1,axis=1)
    else: 
        continue 

c:\anaconda\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [31]:
#validate
#print np.unique(test1[[ 'HOUSENO']])
print np.unique(test1[[ 'PREFIX']])
print np.unique(test1[[ 'STRNAME']])
print np.unique(test1[[ 'STRTYPE']])
print np.unique(test1[[ 'SUFFIX']])
print np.unique(test1[[ 'UNIT_BLD']])
print np.unique(test1[[ 'CITY']])
print np.unique(test1[[ 'STATE']])
print np.unique(test1[[ 'ZIP']])

[nan 'E' 'S' 'SE' 'W']
['1/2 NADEN' '100' '101' '104' '106' '110' '112' '113' '114' '116' '117'
 '119' '120' '121' '122' '123' '124' '128' '131' '132' '133' '134' '138'
 '139' '140' '141' '143' '144' '147' '150' '151' '152' '153' '154' '155'
 '192' '196' '197' '2' '202' '204' '208' '212' '218' '219' '220' '222'
 '227' '228' '229' '238' '240' '241' '242' '243' '244' '245' '246' '248'
 '250' '251' '252' '253' '255' '259' '260' '261' '264' '266' '267' '269'
 '270' '272' '273' '276' '277' '278' '280' '284' '292' '293' '294' '295'
 '297' '38' '43' '5' '72' '76' '84' '90' '91' '92' '93' '94' '96' '97' '98'
 '99' 'ALDER' 'CENTRAL' 'CLEARVIEW' 'FRAGER' 'GUIBERSON' 'HAZEL'
 'KENSINGTON' 'KENT KANGLEY' 'MAPLE' 'MILITARY' 'NADEN' 'PACIFIC' 'REITEN'
 'SCENIC' 'SMITH' 'STATE' 'TITUS' 'VETERANS' 'WASHINGTON' 'WOODFORD']
['AVE' 'CT' 'DR' 'HWY' 'LN' 'PL' 'POINT' 'RD' 'ST' 'WAY']
['' 'N' 'S' 'SE']
[nan '' '#A' 'SPC21' 'SPC7']
['']
['']
['']


c:\anaconda\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [32]:
test1['ZIP'] = test1['Site ZipCode']

In [33]:
np.unique(test1['ZIP'])

array([ 98030.,  98031.,  98032.,  98042.,  98055.,  98092.,     nan,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     nan,     nan,     nan,     nan,     nan,     nan,
           nan,     nan,     nan])

# Drop Commercial 

In [44]:
drop_index = []
for i in range(len(test1)):
    des = test1[i:i+1]['Work Description'].tolist()[0].split()
    if 'COMMERCIAL' in des:
        print i 
        for j in range(len(des)):
            if des[j] == 'COMMERCIAL':
                print des[j:j+9]
        drop_index = drop_index + [i]
        

10
['COMMERCIAL', 'CONSTRUCTION,', 'BUILDING,', 'PLUMBING,', 'MECHANICAL.', '***Electrical', 'work', 'to', 'be']
11
['COMMERCIAL', 'BUILDING,', 'BUILDING,', 'PLUMBING,', 'MECHANICAL.', '***Electrical', 'work', 'to', 'performed']
22
['COMMERCIAL', 'ALTERATION,', 'CHANGE', 'OF', 'USE,', 'BUILDING', 'ONLY.', '***Mechanical,', 'plumbing,']
23
['COMMERCIAL', 'ALTERATION/CHANGE', 'OF', 'USE,', 'BUILDING', 'AND', 'PLUMBING', '***Mechanical', 'and']
24
['COMMERCIAL', 'ALTERATION', 'AND', 'CHANGE', 'OF', 'USE.', 'BUILDING', 'ONLY,', '***']
27
['COMMERCIAL', 'ALTERATION,', 'BUILDING', 'AND', 'PLUMBING', 'ONLY.', '***Mechanical', 'and', 'electrical']
29
['COMMERCIAL', 'ACCESSORY', 'BUILDING.', '329', 'SF', '-', 'PUMP', 'STATION', 'CONSTRUCT']
30
['COMMERCIAL', 'ACCESSORY', 'BUILDING.', '266', 'SF', '-', 'ELECTRICAL', 'SHELTER', 'CONSTRUCT']
31
['COMMERCIAL', 'ACCESSORY', 'BUILDING,', 'BUILDING', 'ONLY..', '513', 'SF', '-', 'GENERATOR']
36
['COMMERCIAL', 'ACCESSORY', 'BUILDING', 'FOR', 'EMPLOYEE',

In [45]:
drop_index

[10, 11, 22, 23, 24, 27, 29, 30, 31, 36, 53]

In [46]:
test2 = test1.drop(test1.index[drop_index]).reset_index(drop=True)

# TYPE & PS

WARNING: 'TYPE', 'PS' ARE VERY DIFFICULT TO CODE, will NEED cross exam this step via excel manually in the end. 

Step1: get MF/Mobile home unit number (###)
* find unit number from work description
* confirm MF numbers from structure type information, permit activities infor, or other available informations

Step2: get SF unit number (1, -1)
* detect if it is new
* detect if it is demolished 
* detect if it is additional dewell unit - depends on new structure type to jusdge unit#

In [47]:
test2 = test1

In [48]:
test2['UNITS'] = ''

In [49]:
print np.unique(test2['Structure Type'])
print np.unique(test2['Permit Activity'])
print "----------------check correlation---------------------"
print np.unique(test2[(test2['Structure Type'] == 'ADU')]['Permit Activity'])
print np.unique(test2[(test2['Structure Type'] == 'MF')]['Permit Activity'])
print np.unique(test2[(test2['Structure Type'] == 'MH')]['Permit Activity'])
print np.unique(test2[(test2['Structure Type'] == 'OTHER')]['Permit Activity'])
print np.unique(test2[(test2['Structure Type'] == 'SF')]['Permit Activity'])

['ADU' 'MF' 'MH' 'Other' 'SF']
[nan 'Addition' 'New']
----------------check correlation---------------------
['Addition']
['New']
['New']
[]
['Addition' 'New']


# -----> Step 1

In [50]:
# find unit number from work description
units_index = []
for i in range(len(test2)):
    des = test2[i:i+1]['Work Description'].tolist()[0].split()
    if 'UNITS' in des:
        print i 
        for j in range(len(des)):
            if des[j] == 'UNITS':
                j = max(0, j-4)
                print des[j:j+10]
        units_index = units_index + [i]

    else: 
        if 'UNITS:' in des:
            print i 
            for j in range(len(des)):
                if des[j] == 'UNITS:':
                    j = max(0, j-4)
                    print des[j:j+10]
            units_index = units_index + [i]

3
['PODIUM', '(4+1)', '154', 'DWELLING', 'UNITS', 'SQUARE', 'FOOTAGE:', '1ST', 'FLOOR', '-']
4
['STORIES:', '4.', 'NUMBER', 'OF', 'UNITS:', '28', 'SQUARE', 'FOOTAGE:', '1ST', 'FLOOR:']
5
['garage).', 'NUMBER', 'OF', 'DWELLING', 'UNITS:', '2.', 'NUMBER', 'OF', 'GARAGE', 'BAYS:']
6
['3.', 'NUMBER', 'OF', 'DWELLING', 'UNITS:', '2.', 'NUMBER', 'OF', 'GARAGE', 'BAYS:']
7
['3.', 'NUMBER', 'OF', 'DWELLING', 'UNITS:', '2.', 'NUMBER', 'OF', 'GARAGE', 'BAYS:']
8
['3.', 'NUMBER', 'OF', 'DWELLING', 'UNITS:', '2.', 'NUMBER', 'OF', 'GARAGE', 'BAYS:']
9
['GROUP:', 'R-2.', 'NUMBER', 'OF', 'UNITS:', '32', 'SQUARE', 'FOOTAGE:', '1ST', 'FLOOR:']
12
['R-2.', 'NUMBER', 'OF', 'DWELLING', 'UNITS:', '16', '2012', 'IBC.', '2012', 'WSEC.']
13
['R-2.', 'NUMBER', 'OF', 'DWELLING', 'UNITS:', '14', '2012', 'IBC.', '2012', 'WSEC.']
14
['R-2.', 'NUMBER', 'OF', 'DWELLING', 'UNITS:', '16', '2012', 'IBC.', '2012', 'WSEC.']
503
['OVER', 'BASEMENT)', 'NUMBER', 'OF', 'UNITS:', '4', 'SQUARE', 'FOOTAGE:', '1ST', 'FLOOR:']
50

In [51]:
units_index

[3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 503, 504, 505, 506, 507, 508, 509, 510, 511]

In [52]:
################### hard coding required {unit index : number of unit } ##########################
units_index_dict = {3: 154, 4:28, 5:2, 6:2, 7:2, 8:2, 9:32, 12:16, 13:14, 14:16, 
                    503:4, 504:2, 505:2, 506:2, 507:4, 508:4, 509:4, 510:4, 511:4}

#fill in the unit number found at work descriptions 
for i in units_index:
    test2.loc[i,'UNITS']= units_index_dict[i]
    test2.loc[i,'PS']= 0

In [120]:
#comfirm if we got everything for multi-family 

In [61]:
# Validate: based on structure type and permit activity, to check if there is any MF permits been left behind 
print len(units_index_dict)
print len(test2[test2['Structure Type'] == 'MF'])
print len(test2[(test2['Structure Type'] == 'MF') & (test2['Permit Activity'] == 'New')])

19
21
21


In [71]:
for i in test2[(test2['Structure Type'] == 'MF') & (test2['Permit Activity'] == 'New')][7:9]['Work Description']: 
    print i 

NEW COMMERCIAL CONSTRUCTION, BUILDING, PLUMBING, MECHANICAL. ***Electrical work to be performed under separate permits. ***  GrandView Building K.  SCOPE OF WORK: Construct six-story aparbment building containing 49 dwelling units.  Structure is a "platform" building with one-story of 1-A construction below a three-hour-rated horizontal assembly, with five-stories of V-A construction above.  See 2012 IBC 510.2.  See WAamended IBC 504.3 (for addition of extra story with pressurized interior exit stairways).  GRANDVIEW BUILDING K.  

R-2, 1-A, V-A, fully sprinkled, NFPA 13 throughout.   Approved sprinkler plan to be on site at inspections.  

Structural analysis: Coughlin Porter Lundeen, S142046-03, 206-343-0460, Christopher Seth Duvall, S.E. 39843
Architect: GGLO, 206-467-5828, James Bradley registered architect 7229.
Geotechnal report: Earth Solutions, job #2918.01, 425-440-4704
Testing Agency for special inspection: Krazen & Associates, 253-939-2500

Deferred Submittals and Revisions:

If the numbers above are all same, perfectly match! No need to do futhure investigation 

# ------> step2

In [72]:
print len(test2[test2['Structure Type'] == 'SF'])
print len(test2[(test2['Structure Type'] == 'SF') & (test2['Permit Activity'] == 'New')])

431
402


In [73]:
test2[(test2['Structure Type'] == 'SF') & (test2['Permit Activity'] == 'New')]['UNITS'] = 1
test2[(test2['Structure Type'] == 'SF') & (test2['Permit Activity'] == 'New')]['PS'] = 0

c:\anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
c:\anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [74]:
print len(test2[test2['Structure Type'] == 'SF'])
print len(test2[(test2['Structure Type'] == 'SF') & (test2['Permit Activity'] == 'Addition')])

431
29


In [75]:
# find the situation from work description
units_index = []
for i in range(len(test2)):
    des = test2[i:i+1]['Work Description'].tolist()[0].split()
    if 'UNITS' in des:
        print i 
        for j in range(len(des)):
            if des[j] == 'UNITS':
                j = max(0, j-4)
                print des[j:j+10]
        units_index = units_index + [i]

3
['PODIUM', '(4+1)', '154', 'DWELLING', 'UNITS', 'SQUARE', 'FOOTAGE:', '1ST', 'FLOOR', '-']
510
['CONSTRUCT', 'TOWNHOUSE', 'WITH', '4', 'UNITS', 'AND', '2', 'HOUR', 'SEPARATION', 'WALLS']
['HOUR', 'SEPARATION', 'WALLS', 'BETWEEN', 'UNITS', '(BLDG.', 'B)', 'TYPE', 'OF', 'CONSTRUCTION:VB']
511
['CONSTRUCT', 'TOWNHOUSE', 'WITH', '4', 'UNITS', 'AND', '2', 'HOUR', 'SEPARATION', 'WALLS']
['HOUR', 'SEPARATION', 'WALLS', 'BETWEEN', 'UNITS', '(BLDG.', 'A)', 'TYPE', 'OF', 'CONSTRUCTION:VB']


In [176]:
test[]

Permit ID#                              Project Name    Tax PID#  \
64   CNST-2161293        CAREPOINTE AFH - GARAGE CONVERSION  8700110030   
65   CNST-2170243  ALPINE CARE LODGE AFH -GARAGE CONVERSION  7970800120   
66   CNST-2170478    NORDTROM RESIDENCE - GARAGE CONVERSION  8700100080   
67   CNST-2171323    ZAUTADZE RESIDENCE - GARAGE CONVERSION  5530350110   
68   CNST-2172574        ORJI RESIDENCE - GARAGE CONVERSION  2722049074   
468  CNST-2132403                HEALD RESIDENCE - ADDITION  6187100320   
469  CNST-2152707                  THOMPSON RESIDENCE - BPM  3728800165   
470  CNST-2162052            THUY H LE RESIDENCE - ADDITION  2122059119   
471  CNST-2162623                KUMAR RESIDENCE - ADDITION  1922059010   
472  CNST-2162388                BEARD RESIDENCE - ADDITION  5470110160   
473  CNST-2162931               NELSON RESIDENCE - ADDITION  8955800045   
474  CNST-2162837                OTANI RESIDENCE - ADDITION  0722059221   
475  CNST-2163302                JONES RESIDENCE - ADDITION  1786700150   
476  CNST-2163980               CARTER RESIDENCE - ADDITION  4050800170   
477  CNST-2161402             ANDERSON RESIDENCE - ADDITION  5466400080   
478  CNST-2154606                WOLDE RESIDENCE - ADDITION  1310990020   
479  CNST-2170638                 KOPP RESIDENCE - ADDITION  5469501710   
480  CNST-2171377              JOHNSON RESIDENCE - ADDITION  3522059166   
481  CNST-2172211              FARWELL RESIDENCE - ADDITION  9183701295   
482  CNST-2170958                 DIAZ RESIDENCE - ADDITION  5467900070   
483  CNST-2171771                LIANG RESIDENCE - ADDITION  1786630240   
484  CNST-2172629                SINGH RESIDENCE - ADDITION  1263400010   
485  CNST-2172825              DELGATO RESIDENCE - ADDITION  6667100530   
486  CNST-2173140               BOWSER RESIDENCE - ADDITION  1091500270   
487  CNST-2172166              WOLCOTT RESIDENCE - ADDITION  8914000045   
488  CNST-2171905             VANGAVER RESIDENCE - ADDITION  5469502840   
489  CNST-2173390              NICHOLS RESIDENCE - ADDITION  3422059212   
490  CNST-2173121             HIDDEN RIDGE LOT C - ADDITION  3222059132   
491  CNST-2174447            MCPHERREN RESIDENCE - ADDITION  2222059068   

               Site Address  Site ZipCode   Issue Date   Final Date Status  \
64         11308 SE 269 ST          98030   05/02/2016   01/12/2017   DONE   
65          11206 S 266 ST          98030   01/25/2017   01/00/1900   OPEN   
66       26700  110 AVE SE          98030   03/16/2017   05/23/2017   DONE   
67         12022 SE 269 PL          98030   04/26/2017   01/00/1900   OPEN   
68           3208 S 259 PL          98032   09/15/2017   01/00/1900   OPEN   
468      23603  138 AVE SE          98042   11/21/2014   04/19/2017   DONE   
469        14514 SE 266 ST          98042   08/28/2015   01/17/2017   DONE   
470        12019 SE 248 ST          98030   06/22/2016   08/09/2017   DONE   
471          9411 S 241 ST          98030   07/28/2016   01/23/2017   DONE   
472       26007  147 PL SE          98042   08/02/2016   03/28/2017   DONE   
473        14621 SE 267 PL          98042   09/09/2016   06/16/2017   DONE   
474         21450  94 PL S          98031   09/30/2016   05/30/2017   DONE   
475       25215  138 PL SE          98042   11/08/2016   08/29/2017   DONE   
476        14224 SE 270 PL          98042   01/09/2017   01/00/1900   OPEN   
477        13616 SE 273 CT          98042   01/09/2017   01/00/1900   OPEN   
478         25413  38 PL S          98032   01/17/2017   09/13/2017   DONE   
479      24731  144 AVE SE          98042   04/20/2017   01/00/1900   OPEN   
480        15205 SE 278 ST          98042   05/17/2017   11/29/2017   DONE   
481      520  SCENIC WAY S          98030   07/18/2017   01/09/2018   DONE   
482      26215  143 AVE SE          98042   07/31/2017   02/01/2018   DONE   
483        13927 SE 238 PL          98042   08/07/2017   10/10/2017   DONE   
484        12524 SE 264 ST            Na

# NOTES, 2, 3, 4, 5

In [76]:
test2 = test1

In [77]:
for c in ['NOTES', 'NOTES2', 'NOTES3', 'NOTES4', 'NOTES5']:
    test2[c] = ''

In [78]:
test2.columns

Index([u'Permit ID#', u'Project Name', u'Tax PID#', u'Site Address',
       u'Site ZipCode', u'Issue Date', u'Final Date', u'Status', u'# of Units',
       u'# of Buildings', u'Est. Value', u'Condo', u'Lot Area/Size',
       u'Zoning Code', u'Work Description', u'Structure Type',
       u'Permit Activity', u'Land Use', u'SORT', u'HOUSENO', u'PREFIX',
       u'STRNAME', u'STRTYPE', u'SUFFIX', u'UNIT_BLD', u'CITY', u'STATE',
       u'ZIP', u'UNITS', u'PS', u'NOTES', u'NOTES2', u'NOTES3', u'NOTES4',
       u'NOTES5'],
      dtype='object')

In [97]:
test2['NOTES'] = test2['Project Name']
#test2['NOTES2'] = test2['Lot Area/Size']
test2['NOTES3'] = test2['Structure Type']
test2['NOTES5'] = test2['Status']
test2['NOTES6'] = ''
test2['NOTES7'] = ''
#test2['NOTES6'] = test2['Type'] # just reserve some info from raw data
#test2['NOTES7'] = test2['Sub-Type'] # reserve some info from raw data 
for i in range(len(test2)):
    work = test2['Work Description'][i]
    test2['NOTES4'][i] = work[:50]


c:\anaconda\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Extra information to carry over

The first two digits indicate the project year. The second two digits indicate the county in which the permit is located by abbreviated FIPS county code. The next five digits (included since 2000) indicate the jurisdiction of permit issue by FIPS place code. The last five digits serve as an automatic counter.

In [99]:
test3 = test2

In [100]:
test3['PERMITNO'] = test3['Permit ID#']

In [101]:
test3['PSRCIDN'] = '' # no data 
test3['PIN'] = '' # no data 
test3['TYPE'] = ''
test3['PS'] = ''
test3['MULTIREC'] = ''
test3['LANDUSE'] = ''
test3['CONDO'] = ''
test3['ZONING'] = ''
test3['LOTSIZE'] = '' 
test3['PIN_PARENT'] = ''
test3['BLDGS'] = 1
test3['FINALED'] = test3['Final Date']
test3['ISSUED'] = test3['Issue Date']
test3['STATUS'] = test3['Status']
test3['VALUE'] = test3['Est. Value']
test3['ADDRESS'] = test3['Site Address']

In [92]:
test3.columns

Index([u'Permit ID#', u'Project Name', u'Tax PID#', u'Site Address',
       u'Site ZipCode', u'Issue Date', u'Final Date', u'Status', u'# of Units',
       u'# of Buildings', u'Est. Value', u'Condo', u'Lot Area/Size',
       u'Zoning Code', u'Work Description', u'Structure Type',
       u'Permit Activity', u'Land Use', u'SORT', u'HOUSENO', u'PREFIX',
       u'STRNAME', u'STRTYPE', u'SUFFIX', u'UNIT_BLD', u'CITY', u'STATE',
       u'ZIP', u'UNITS', u'PS', u'NOTES', u'NOTES2', u'NOTES3', u'NOTES4',
       u'NOTES5', u'PERMITNO', u'PSRCIDN', u'PIN', u'TYPE', u'MULTIREC',
       u'LANDUSE', u'CONDO', u'ZONING', u'LOTSIZE', u'PIN_PARENT', u'BLDGS',
       u'FINALED', u'ISSUED', u'STATUS', u'VALUE'],
      dtype='object')

# Template 

In [102]:
final_col = list(temp.columns.values) + ['CITY', 'STATE'] 

In [103]:
final_col

['PSRCIDN',
 'PERMITNO',
 'SORT',
 'MULTIREC',
 'PIN',
 'ADDRESS',
 'HOUSENO',
 'PREFIX',
 'STRNAME',
 'STRTYPE',
 'SUFFIX',
 'UNIT_BLD',
 'ZIP',
 'ISSUED',
 'FINALED',
 'STATUS',
 'TYPE',
 'PS',
 'UNITS',
 'BLDGS',
 'LANDUSE',
 'CONDO',
 'VALUE',
 'ZONING',
 'NOTES',
 'NOTES2',
 'NOTES3',
 'NOTES4',
 'NOTES5',
 'NOTES6',
 'NOTES7',
 'LOTSIZE',
 'PIN_PARENT',
 'CITY',
 'STATE']

In [104]:
test3.columns

Index([u'Permit ID#', u'Project Name', u'Tax PID#', u'Site Address',
       u'Site ZipCode', u'Issue Date', u'Final Date', u'Status', u'# of Units',
       u'# of Buildings', u'Est. Value', u'Condo', u'Lot Area/Size',
       u'Zoning Code', u'Work Description', u'Structure Type',
       u'Permit Activity', u'Land Use', u'SORT', u'HOUSENO', u'PREFIX',
       u'STRNAME', u'STRTYPE', u'SUFFIX', u'UNIT_BLD', u'CITY', u'STATE',
       u'ZIP', u'UNITS', u'PS', u'NOTES', u'NOTES2', u'NOTES3', u'NOTES4',
       u'NOTES5', u'PERMITNO', u'PSRCIDN', u'PIN', u'TYPE', u'MULTIREC',
       u'LANDUSE', u'CONDO', u'ZONING', u'LOTSIZE', u'PIN_PARENT', u'BLDGS',
       u'FINALED', u'ISSUED', u'STATUS', u'VALUE', u'NOTES6', u'NOTES7',
       u'ADDRESS'],
      dtype='object')

In [110]:
df = test3[final_col]


In [111]:
os.getcwd()

'C:\\Users\\Angela\\Documents\\GitHub\\Latest_Github\\permits\\scripts'

In [112]:
df.to_csv(r'J:\Projects\Permits\17Permit\data\2working\after_process_KENT.csv')